In [1]:
from math import pow
import random
import time
from statistics import mean
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def backward_substitution(L, b):
    x = np.zeros_like(b)
    for i in range(len(L)-1, -1, -1):
        s = 0
        for j in range (i+1, len(L)):
            s += A[i, j]*x[j]
        x[i] = (b[i] - s)/A[i, i]
    return x

In [3]:
A = np.array([[1, 0, 0], [0, 2, 0], [0, 0, 3]])
b = [2, 4, 3]
print(backward_substitution(A, b))
print(np.linalg.solve(A, b))

[2 2 1]
[2. 2. 1.]


In [4]:
def forward_substitution(L, b):
    x = np.zeros_like(b)
    for i in range(len(L)):
        s = 0
        for j in range(i):
            s += L[i, j]*x[j]
        x[i] = (b[i]-s)/L[i, i]
    return x

In [5]:
A = np.array([[1, 0, 0], [-1, 3, 0], [0, -3, 3]])
b = [4, 2, -3]
print(forward_substitution(A, b))
print(np.linalg.solve(A, b))

[4 2 1]
[4. 2. 1.]


In [6]:
def random_lower_triangular_matrix(m, n, random_range = (1, 10)):
    A = []
    for i in range(m):
        wiersz = []
        for j in range(n):
            if i<=j:
                wiersz.append(random.randint(random_range[0], random_range[1]))
            else:
                wiersz.append(0)
        A.append(wiersz)
    return np.array(A)

In [7]:
def gaussian_elimination(A, b):
    n = A.shape[0]
    A_b = np.c_[A, b]
    for i in range(n):
        for k in range(i, n):
            if abs(A_b[k, i])>=1/(2**16):
                A_b[[k, i]] = A_b[[i, k]]
                pass
        if k == n-1 and abs(A_b[k, i])<1/(2**16):
                break
        
        for j in range(i+1, n):
            const = -A_b[j, i]/A_b[i, i]
            A_b[j, i:] = A_b[j, i:] + const*A_b[i, i:]
    return A_b[:,:n], A_b[:, n]

In [8]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
b = np.array([1, 2, 3])
A = np.array([[1, 2, 0], [0, 5, 0], [0, 8, 0]])
b = np.array([2, 2, 3,])
U, c = gaussian_elimination(A, b)
print('U=', U, '\nc=', c)

U= [[1 2 0]
 [0 5 0]
 [0 8 0]] 
c= [2 2 3]


In [9]:
def gaussian_det(A):
    n = A.shape[0]
    ilosc_translacji = 0
    for i in range(n):
        if abs(A[i, i])<1/(2**16):
            ilosc_translacji += 1
            for k in range(i, n):
                if abs(A[k, i])>=1/(2**16):
                    A[[k, i]] = A[[i, k]]
                    pass
            if k == n-1 and abs(A[k, i])<1/(2**16):
                return 0
        
        for j in range(i+1, n):
            const = -A[j, i]/A[i, i]
            A[j, i:] = A[j, i:] + const*A[i, i:]
    det = 1
    for i in range(n):
        det = det*A[i, i]
    return (-1)**ilosc_translacji*det

In [10]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
A = np.array([[1, 0, 1], [0, 1, 0], [0, 1, 0]])
print('det A =', gaussian_det(A))

det A = 0


In [11]:
def gaussian_inv(A):
    n = A.shape[0]
    ilosc_translacji = 0
    A_id = np.c_[A, np.identity(n)]
    for i in range(n):
        for k in range(i, n):
            if abs(A_id[k, i])>=1/(2**16):
                A_id[[k, i]] = A_id[[i, k]]
                pass
        if k == n-1 and abs(A[k, i])<1/(2**16):
            raise Exception("Macierz nie jest odwracalna.")
        A_id[i, i:] = (1/A_id[i, i])*A_id[i, i:]
        
        for j in range(n):
            if i != j:
                const = -A_id[j, i]
                A_id[j, i:] = A_id[j, i:] + const*A_id[i, i:]
    return A_id[:, n:]

In [12]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
A = np.array([[0, 0], [0, 1]])
A = np.array([[1, 0, 0], [0, 1, 0], [1, 1, 0]])
print('det A =', gaussian_inv(A))
print(np.linalg.inv(A))

Exception: Macierz nie jest odwracalna.

In [151]:
#Kod działający

def gaussian_rank(A):
    if A.shape[0]>A.shape[1]:
        A = A.T
    n = A.shape[0]
    for i in range(n):
        for j in range(i, A.shape[1]):
            for k in range(i, n):
                if abs(A[k, j])>=1/(2**16):
                    A[:, [i, j]] = A[:, [j, i]]
                    A[[k, i]] = A[[i, k]]
                    break
            break
        if j == A.shape[1]-1 and abs(A[i, i])<1/(2**16):
            return i
        
        for l in range(i+1, n):
            const = -A[l, i]/A[i, i]
            A[l, i:] = A[l, i:] + const*A[i, i:]
    return n

In [149]:
#Kod działający

def gaussian_rank(A):
    if A.shape[0]>A.shape[1]:
        A = A.T
    n = A.shape[0]
    for i in range(n):
        for j in range(i, A.shape[1]):
            if np.any(A[i:, j]):
                A[:, [i, j]] = A[:, [j, i]]
                for k in range(i, n):
                    if abs(A[k, i])>=1/(2**16):
                        A[[k, i]] = A[[i, k]]
                        break
                break
            if j == A.shape[1]-1 and abs(A[i, j])<1/(2**16):
                return i
        
        for l in range(i+1, n):
            const = -A[l, i]/A[i, i]
            A[l, i:] = A[l, i:] + const*A[i, i:]
    return n

In [152]:
A = np.array([[1, 0, 1, 1, 0, 0], 
              [0, 0, 0, 0, 0, 1],
              [1, 0, 1, 1, 0, 0], 
              [0, 0, 0, 1, 0, 0]])
#A = np.array([[5, 7], [0, 0]])
#A = np.array([[1, 2], [3, 5], [4, 7], [6, 8]])
#A = np.array([[1, 2, 3, 4], [2, 4, 6, 8]])
print('rank A =', gaussian_rank(A))
print('Pakiet linalg uważa, ze odpowiedź to', np.linalg.matrix_rank(A))

rank A = 1
Pakiet linalg uważa, ze odpowiedź to 3
